In [1]:
customer = sc.textFile('/FileStore/tables/customer.txt')
orders = sc.textFile('/FileStore/tables/orders.txt')

In [2]:
customer.count()

Out[ 2 ]: 12435

In [3]:
orders.count()

Out[ 4 ]: 68883

In [4]:
# analyzing the content of both the datasets

In [5]:
customer.take(3)

Out[ 6 ]: 
['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725']

In [6]:
orders.take(3)

Out[ 7 ]: 
['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE']

In [7]:
# Question 1 - Top 5 states with most of cancelled orders

In [8]:
#fetch out (customer_id, state) tuple from customer rdd
customer = customer.map(lambda x:(x.split(',')[0], x.split(',')[-2]))

In [9]:
customer.top(2)

Out[ 22 ]: [('9999', 'NM'), ('9998', 'PR')]

In [10]:
# filter canceled orders out of orders dataset

In [11]:
canceled_orders = orders.filter(lambda x: x.split(',')[-1]=='CANCELED')

In [12]:
# keep only (customer_id,order_id) from canceled orders rdd
canceled_orders = canceled_orders.map(lambda x: (x.split(',')[-2], x.split(',')[0]))

In [13]:
# (customer_id,order_id) from canceled orders rdd
canceled_orders.top(3)

Out[ 20 ]: [('9998', '59713'), ('9993', '21390'), ('9988', '42835')]

In [14]:
# do join of canceled orders with that of customer datset

In [15]:
cust_order = customer.join(canceled_orders)

In [16]:
cust_order.top(3)

Out[ 24 ]: 
[('9998', ('PR', '59713')),
 ('9993', ('OK', '21390')),
 ('9988', ('PR', '42835'))]

In [17]:
# using map form rdd having tuple of type(state,1)

In [18]:
state_count = cust_order.map(lambda x: (x[1][0], 1))

In [19]:
state_count.top(3)

Out[ 27 ]: [('WV', 1), ('WV', 1), ('WI', 1)]

In [20]:
# do a reduceByKey operation while adding the values of the same keys

In [21]:
state_wise_canceled_count = state_count.reduceByKey(lambda x,y: x+y)

In [22]:
state_wise_canceled_count.count()

Out[ 31 ]: 41

In [23]:
#sort the rdd of tuples in descending order of values

In [24]:
state_wise_canceled_count = state_wise_canceled_count.sortBy(lambda x: -x[1])

In [25]:
state_wise_canceled_count.collect()

Out[ 33 ]: 
[('PR', 550),
 ('CA', 242),
 ('NY', 87),
 ('TX', 69),
 ('IL', 54),
 ('FL', 37),
 ('OH', 33),
 ('NJ', 32),
 ('AZ', 28),
 ('PA', 27),
 ('GA', 25),
 ('MI', 24),
 ('NC', 23),
 ('MA', 19),
 ('MD', 18),
 ('OR', 17),
 ('VA', 12),
 ('CO', 12),
 ('LA', 12),
 ('TN', 11),
 ('HI', 11),
 ('NV', 10),
 ('MO', 10),
 ('NM', 9),
 ('WI', 8),
 ('CT', 8),
 ('MN', 7),
 ('DC', 6),
 ('SC', 5),
 ('WA', 4),
 ('KS', 3),
 ('OK', 3),
 ('DE', 2),
 ('WV', 2),
 ('KY', 2),
 ('MT', 1),
 ('IN', 1),
 ('ID', 1),
 ('AR', 1),
 ('UT', 1),
 ('ND', 1)]

In [26]:
state_wise_canceled_count.take(5)

Out[ 35 ]: [('PR', 550), ('CA', 242), ('NY', 87), ('TX', 69), ('IL', 54)]